This project was inspired by this university paper written back in 2015. I tried finding the datasets online but was unable to, so I decided to replicate the work done in this paper. 

http://uu.diva-portal.org/smash/get/diva2:846981/FULLTEXT01.pdf 

In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from matplotlib import pyplot as plt
import seaborn 

In [2]:
prefix = "http://www.perverted-justice.com"
webpage_response = requests.get('http://www.perverted-justice.com/?archive=byUserVotes')

In [3]:
#stores the content of webpage we're scraping for links
webpage = webpage_response.content
soup = BeautifulSoup(webpage, "lxml") #lxml is faster than html.parser"

In [4]:
#stores all 'a' tags with attribute "id" equal to "pedoLink"
pedo_links = soup.findAll("a", {"id": "pedoLink"})
count = len(pedo_links)
print(count)

627


Bingo! That's the number of pedo links provided on website! Making progress.

In [5]:
links = []
# We take the href part of the a tag, then marry that to the prefix, which is the base part for each link
for a in pedo_links:
    links.append(prefix+a["href"])

In [6]:
pedo_data = {}

metadata_list = []
# I want to pull all the span tags with attribute inText
# Then I want to pull the entire conversation 
for link in links:
    subpage = requests.get(link)
    subsoup = BeautifulSoup(subpage.content, "lxml")
    metadata= subsoup.findAll("span", {"class": "inText"}) #finds all the span tags that have class = inText
    str1 = ''.join(str(e) for e in metadata) #converts metadata into strings
    temp_list = re.compile('<span class="inText">(.*?)</span>', re.DOTALL).findall(str1) #pulls only the text between the span tags
    if len(temp_list) == 6:
        #From looking at the data, I noticed there were 5 rows where the data was shifted by one place as there was 
        #no age provided for these individuals. I chose to insert age 26 as this is the most frequent age.
        temp_list.insert(1, '26')
        print(temp_list)
    metadata_list.append(temp_list)

['Loren', '26', 'magic4isl2002', 'Poway, San Diego, California', 'Tiffanyjoy2cute', '13 year old', 'girl']
['Jacob', '26', 'jacob_cason2003', 'Nashville, Georgia', 'sassicassi93', '13 year old', 'girl']
['William', '26', 'welfare_isforwhitefolks', ', North Carolina', 'jewelykool', '14 year old', 'girl']
['Timothy', '26', 'baddboyysweetheart', 'Beaver Dam, Wisconsin', 'lil_suzie_qt', '13 year old', 'girl']
['Brian Richard Williams', '26', 'bmichigan69', 'Clarkston, Michigan', 'yayitskc95', '14 year old', 'girl']


In [7]:
#Convert list of list to dataframe
df = pd.DataFrame(metadata_list, columns=["pedo_name", "pedo_age", "pedo_userid", "city_state", "victim_userid", "victim_age", "victim_gender"])
print(df.head())

                        pedo_name   pedo_age                pedo_userid  \
0                      Paul Short         34  fleet_captain_jaime_wolfe   
1               Michael J. Coffey         54                DavieWants2   
2                             Joe         28              vamale_692005   
3                   Kalonji Woods         21               kalowoodsman   
4  Keith Prather, Timothy Thompso  44 and 26             thenewperson62   

                            city_state   victim_userid   victim_age  \
0                    Chicago, Illinois      sadlilgrrl  13 year old   
1             Suffolk County, New York   SnapShotDeath  14 year old   
2                 Alexandria, Virginia    sweet_erin78  14 year old   
3                 Radford VA, Virginia            keri     underage   
4  Conroe / Willis / Montgomery, Texas  jackies2cool4u  13 year old   

  victim_gender  
0          girl  
1           boy  
2          girl  
3          girl  
4          girl  


In [8]:
#Break out seperate column for state and city
new = df["city_state"].str.split(",", expand=True)
print(new)

                                0            1     2
0                         Chicago     Illinois  None
1                  Suffolk County     New York  None
2                      Alexandria     Virginia  None
3                      Radford VA     Virginia  None
4    Conroe / Willis / Montgomery        Texas  None
..                            ...          ...   ...
622              Saint Petersburg      Florida  None
623                     Coldwater     Michigan  None
624                        Queens     New York  None
625                  Garden Grove   California  None
626                 West Millford   New Jersey  None

[627 rows x 3 columns]


In [9]:
df["city"]= new[0] 
df["state"]= new[1] 

In [10]:
print(df.head())

                        pedo_name   pedo_age                pedo_userid  \
0                      Paul Short         34  fleet_captain_jaime_wolfe   
1               Michael J. Coffey         54                DavieWants2   
2                             Joe         28              vamale_692005   
3                   Kalonji Woods         21               kalowoodsman   
4  Keith Prather, Timothy Thompso  44 and 26             thenewperson62   

                            city_state   victim_userid   victim_age  \
0                    Chicago, Illinois      sadlilgrrl  13 year old   
1             Suffolk County, New York   SnapShotDeath  14 year old   
2                 Alexandria, Virginia    sweet_erin78  14 year old   
3                 Radford VA, Virginia            keri     underage   
4  Conroe / Willis / Montgomery, Texas  jackies2cool4u  13 year old   

  victim_gender                          city      state  
0          girl                       Chicago   Illinois  
1   

In [11]:
#remove the "year old" from age column
new2 = df["victim_age"].str.split(" ",expand=True)
df["child_age"] = new2[0]
print(df.head())

                        pedo_name   pedo_age                pedo_userid  \
0                      Paul Short         34  fleet_captain_jaime_wolfe   
1               Michael J. Coffey         54                DavieWants2   
2                             Joe         28              vamale_692005   
3                   Kalonji Woods         21               kalowoodsman   
4  Keith Prather, Timothy Thompso  44 and 26             thenewperson62   

                            city_state   victim_userid   victim_age  \
0                    Chicago, Illinois      sadlilgrrl  13 year old   
1             Suffolk County, New York   SnapShotDeath  14 year old   
2                 Alexandria, Virginia    sweet_erin78  14 year old   
3                 Radford VA, Virginia            keri     underage   
4  Conroe / Willis / Montgomery, Texas  jackies2cool4u  13 year old   

  victim_gender                          city      state child_age  
0          girl                       Chicago   Illin

In [12]:
#Since I broke out the city_state column and have a column just for age, I'm going to drop these two original columns
df = df.drop(['city_state', 'victim_age'], axis=1) #axis=1 specifies this is applied to columns (column operation)

In [13]:
df.groupby('victim_gender').count()

,pedo_name,pedo_age,pedo_userid,victim_userid,city,state,child_age
victim_gender,,,,,,,
boy,52,52,52,52,52,52,52
girl,549,549,549,549,549,549,549


In [14]:
df.groupby('child_age').count()

,pedo_name,pedo_age,pedo_userid,victim_userid,victim_gender,city,state
child_age,,,,,,,
11,2,2,2,2,2,2,2
12,46,46,46,46,46,46,46
12/13,1,1,1,1,1,1,1
13,332,332,332,332,332,332,332
13/14,1,1,1,1,1,1,1
14,184,184,184,184,184,184,184
15,23,23,23,23,23,23,23
underage,12,12,12,12,12,12,12


In [15]:
underaged = df.loc[df['child_age'] == 'underage']
print(underaged)

                   pedo_name pedo_age                pedo_userid  \
3              Kalonji Woods       21               kalowoodsman   
5            Thomas Humphrey       28               jackman_9682   
162            Tyson Jenkins       35                 ghost27_73   
165  Eduardo Ernesto Guillen       34                 tito_92503   
256                     Ted        33             bigdaddy68iou1   
292         Billy Wayne Rife       38               bwblueeyes04   
300                  Anthony       26  yp_anthony_louisville_284   
314                      Jed       22      ich_bin_der_eggman_67   
337                  Russell       34               cooperisaaca   
343                     Seda       23                  hey_malen   
509                     Jose       21                 jagjen2003   
591            Anurag Tawari       29                indianman76   

         victim_userid victim_gender          city        state child_age  
3                 keri          girl   

In [16]:
#I'm going to use the number in the victim_userid as a hint of the age of the individual
df.loc[5,'child_age'] = 13
df.loc[162,'child_age'] = 14
df.loc[256,'child_age'] = 14
df.loc[337,'child_age'] = 14
df.loc[343,'child_age'] = 13

In [17]:
# Since 13 is by far the most common child_age I'm going to replace underage with 13. 
df['child_age'] = df.child_age.str.replace('underage', '13') 
df['child_age'] = df.child_age.str.replace('12/13', '13') 
df['child_age'] = df.child_age.str.replace('13/14', '13') 

In [18]:
underaged = df.loc[df['child_age'] == 'underage']
print(underaged)

Empty DataFrame
Columns: [pedo_name, pedo_age, pedo_userid, victim_userid, victim_gender, city, state, child_age]
Index: []


In [19]:
df.groupby('child_age').count()

,pedo_name,pedo_age,pedo_userid,victim_userid,victim_gender,city,state
child_age,,,,,,,
11,2,2,2,2,2,2,2
12,46,46,46,46,46,46,46
13,341,341,341,341,341,341,341
14,184,184,184,184,184,184,184
15,23,23,23,23,23,23,23


In [20]:
df.groupby('pedo_age').count()

,pedo_name,pedo_userid,victim_userid,victim_gender,city,state,child_age
pedo_age,,,,,,,
"""28""",1,1,1,1,1,1,1
19,6,6,6,6,6,6,6
20,8,8,7,7,8,8,7
21,25,25,24,24,25,25,24
22,28,28,26,26,28,28,26
23,34,34,34,34,34,34,33
24,30,30,29,29,30,30,29
25,26,26,23,23,26,26,23
26,43,43,43,43,43,43,43


In [21]:
#I'm going to conver the 28 into a string without quotes
df['pedo_age'] = df.pedo_age.str.replace('"28"', '28') 

#I decided to drop this row as it appears two individuals worked together and this was a one off situation. 
df = df.drop(df.index[4])

In [22]:
df.groupby('pedo_age').count()

,pedo_name,pedo_userid,victim_userid,victim_gender,city,state,child_age
pedo_age,,,,,,,
19,6,6,6,6,6,6,6
20,8,8,7,7,8,8,7
21,25,25,24,24,25,25,24
22,28,28,26,26,28,28,26
23,34,34,34,34,34,34,33
24,30,30,29,29,30,30,29
25,26,26,23,23,26,26,23
26,43,43,43,43,43,43,43
27,30,30,29,29,30,30,29


In [23]:
#Checkout datatypes
df.dtypes

pedo_name        object
pedo_age         object
pedo_userid      object
victim_userid    object
victim_gender    object
city             object
state            object
child_age        object
dtype: object

In [24]:
print((df['child_age'].unique()))
df1 = df[df.isna().any(axis=1)]
print(df1.count())

['13' '14' nan '12' '15' '11']
pedo_name        31
pedo_age         31
pedo_userid      31
victim_userid     5
victim_gender     5
city             31
state            31
child_age         0
dtype: int64


In [25]:
df['child_age'] = df['child_age'].fillna('13')

In [26]:
df.groupby('child_age').count()

,pedo_name,pedo_age,pedo_userid,victim_userid,victim_gender,city,state
child_age,,,,,,,
11,2,2,2,2,2,2,2
12,46,46,46,46,46,46,46
13,371,371,371,345,345,371,371
14,184,184,184,184,184,184,184
15,23,23,23,23,23,23,23


In [27]:
print((df['child_age'].unique()))

['13' '14' '12' '15' '11']


In [28]:
#Convert objects to integers
df['pedo_age']=df['pedo_age'].astype(str).astype(int)
df['child_age']=df['child_age'].astype(str).astype(int)

In [29]:
df.dtypes

pedo_name        object
pedo_age          int64
pedo_userid      object
victim_userid    object
victim_gender    object
city             object
state            object
child_age         int64
dtype: object

In [30]:
df.groupby('state').count()

,pedo_name,pedo_age,pedo_userid,victim_userid,victim_gender,city,child_age
state,,,,,,,
AL,1,1,1,1,1,1,1
Alabama,15,15,15,14,14,15,15
Arizona,4,4,4,4,4,4,4
Arkansas,5,5,5,5,5,5,5
CA,5,5,5,4,4,5,5
...,...,...,...,...,...,...,...
Washington,12,12,12,12,12,12,12
West Virginia,1,1,1,1,1,1,1
Wisconsin,21,21,21,21,21,21,21


In [31]:
df.state.unique()

array([' Illinois', ' New York', ' Virginia', ' Oklahoma', ' Washington',
       ' Texas', ' California', ' Georgia', ' Florida', ' Maryland',
       ' Arizona', 'Oklahoma', ' Pennsylvania', ' Arkansas', ' MA',
       ' Missouri', ' Alabama', ' Connecticut', ' Minnesota', ' Oregon',
       ' Rhode Island', ' Michigan', ' Idaho', ' Ohio', ' Mississippi',
       ' Indiana', ' South Carolina', ' Wisconsin', ' Nevada',
       ' Kentucky', ' Colorado', ' Massachusetts', ' WI', ' Kansas',
       ' GA', ' District of Columbia', ' Utah', ' Louisiana', ' WA',
       ' Maine', ' North Dakota', ' New Hampshire', ' MI', ' CA', ' AL',
       ' KY', ' Nebraska', ' San Diego', ' Tennessee', ' New Jersey',
       ' Ok', ' FL', ' TN', ' OK', ' North Carolina', ' Calif.',
       'Indiana', ' OH', ' New Mexico', ' Vermont', ' West Virginia'],
      dtype=object)

In [32]:
df['state'] = df.state.str.replace('WI', 'Wisconsin') 
df['state'] = df.state.str.replace('MA', 'Massachusetts') 
df['state'] = df.state.str.replace('GA', 'Georgia') 
df['state'] = df.state.str.replace('WA', 'Washington') 
df['state'] = df.state.str.replace('MI', 'Michigan') 
df['state'] = df.state.str.replace('CA', 'California') 
df['state'] = df.state.str.replace('AL', 'Alabama') 
df['state'] = df.state.str.replace('KY', 'Kentucky') 
df['state'] = df.state.str.replace('San Diego', 'California') 
df['state'] = df.state.str.replace('Ok', 'Oklahoma') 
df['state'] = df.state.str.replace('FL', 'Florida') 
df['state'] = df.state.str.replace('TN', 'Tennessee') 
df['state'] = df.state.str.replace('OK', 'Oklahoma') 
df['state'] = df.state.str.replace('Calif.', 'California') 
df['state'] = df.state.str.replace('OH', 'Ohio') 
df['state'] = df.state.str.replace('Californiarnia', 'California') 
df['state'] = df.state.str.replace('Oklahomalahoma', 'Oklahoma') 

In [40]:
states = sorted(df['state'].unique())
print(states)

[' Alabama', ' Arizona', ' Arkansas', ' California', ' Colorado', ' Connecticut', ' District of Columbia', ' Florida', ' Georgia', ' Idaho', ' Illinois', ' Indiana', ' Kansas', ' Kentucky', ' Louisiana', ' Maine', ' Maryland', ' Massachusetts', ' Michigan', ' Minnesota', ' Mississippi', ' Missouri', ' Nebraska', ' Nevada', ' New Hampshire', ' New Jersey', ' New Mexico', ' New York', ' North Carolina', ' North Dakota', ' Ohio', ' Oklahoma', ' Oregon', ' Pennsylvania', ' Rhode Island', ' South Carolina', ' Tennessee', ' Texas', ' Utah', ' Vermont', ' Virginia', ' Washington', ' West Virginia', ' Wisconsin', 'Indiana', 'Oklahoma']


In [45]:
states_count = (df.groupby('state').count())
print(states_count)

                       pedo_name  pedo_age  pedo_userid  victim_userid  \
state                                                                    
 Alabama                      16        16           16             15   
 Arizona                       4         4            4              4   
 Arkansas                      5         5            5              5   
 California                  148       148          148            139   
 Colorado                      1         1            1              1   
 Connecticut                  12        12           12             11   
 District of Columbia          1         1            1              1   
 Florida                      41        41           41             39   
 Georgia                      33        33           33             30   
 Idaho                         5         5            5              4   
 Illinois                      9         9            9              9   
 Indiana                      13      

In [37]:
df1 = df[df.isna().any(axis=1)]
print(df1)

                pedo_name  pedo_age                   pedo_userid  \
36               Clifford        33                      FOTOPHIX   
47          No Name Given        30                     test_post   
130                 Ramin        42               sebastian_calif   
188               Rolando        33  marriedandlookingforfun31313   
198                Corye         22                       blaggca   
209                Heidar        61                     mikejardi   
216               Stephen        35              stephen_tanner69   
220                Josiah        27                 hot95cobraguy   
246              Jonathan        20                 johnchess2000   
260        John Adamowski        37                john_adamowski   
279         Robert Graber        25                    ekoplaya20   
301           Joshua Fuhr        21                  edgesoccer22   
318                George        36                broken_empires   
346            Larry Aspy        5

In [ ]:
plt.style.use('ggplot')

x = [' Alabama', ' Arizona', ' Arkansas', ' California', ' Colorado', ' Connecticut', ' District of Columbia', ' Florida', ' Georgia', ' Idaho', ' Illinois', ' Indiana', ' Kansas', ' Kentucky', ' Louisiana', ' Maine', ' Maryland', ' Massachusetts', ' Michigan', ' Minnesota', ' Mississippi', ' Missouri', ' Nebraska', ' Nevada', ' New Hampshire', ' New Jersey', ' New Mexico', ' New York', ' North Carolina', ' North Dakota', ' Ohio', ' Oklahoma', ' Oregon', ' Pennsylvania', ' Rhode Island', ' South Carolina', ' Tennessee', ' Texas', ' Utah', ' Vermont', ' Virginia', ' Washington', ' West Virginia', ' Wisconsin', 'Indiana', 'Oklahoma']
energy = [5, 6, 15, 22, 24, 8]

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, energy, color='green')
plt.xlabel("Energy Source")
plt.ylabel("Energy Output (GJ)")
plt.title("Energy output from various fuel sources")

plt.xticks(x_pos, x)

plt.show()

In [31]:
#Graphs of ages, cities, states
#NLP? - break out words from usernames, sentiment analysis?

In [32]:
#https://stackoverflow.com/questions/57484530/beautifulsoup-find-all-and-get-text for conversation part of website